In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time
import math
from collections import deque, Counter


In [2]:
HAND_COLORS = {"Left": (255, 255, 0), "Right": (255, 0, 255)}
JOINT_COLOR = {"Straight": (0, 255, 0), "Bent": (0, 0, 255)}
VIOLET = (148, 0, 211)

STRAIGHT_THRESH = 160
THUMB_THRESH = 140
SMOOTH_ALPHA = 0.6
GESTURE_HISTORY = 10
GESTURE_CONFIRM = 6
DIST_OK_RATIO = 0.18

MODE_NORMAL = 0
MODE_GESTURE = 1
MODE_ANGLE = 2
mode = MODE_NORMAL


In [3]:
GESTURE_PATTERNS = {
    "Fist": {"Thumb":"Bent","Index":"Bent","Middle":"Bent","Ring":"Bent","Pinky":"Bent"},
    "Open Hand": {"Thumb":"Straight","Index":"Straight","Middle":"Straight","Ring":"Straight","Pinky":"Straight"},
    "Peace ✌️": {"Index":"Straight","Middle":"Straight","Ring":"Bent","Pinky":"Bent","Thumb":"Bent"},
    "Thumbs Up 👍": {"Thumb":"Straight","Index":"Bent","Middle":"Bent","Ring":"Bent","Pinky":"Bent"},
    "Pointing ☝️": {"Index":"Straight","Thumb":"Bent","Middle":"Bent","Ring":"Bent","Pinky":"Bent"},
    "Okay 👌": {"Thumb":"Bent","Index":"Bent","Middle":"Straight","Ring":"Straight","Pinky":"Straight"},
    "Rock 🤘": {"Index":"Straight","Pinky":"Straight","Thumb":"Bent","Middle":"Bent","Ring":"Bent"},
    "Call Me 🤙": {"Thumb":"Straight","Pinky":"Straight","Index":"Bent","Middle":"Bent","Ring":"Bent"},
}


In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

In [5]:
def calculate_angle(a, b, c):
    a = np.array(a); b = np.array(b); c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(np.degrees(radians))
    return 360 - angle if angle > 180 else angle

def finger_state(finger_name, angle):
    thresh = THUMB_THRESH if finger_name == "Thumb" else STRAIGHT_THRESH
    return "Straight" if angle > thresh else "Bent"

def smooth_position(key, x, y, positions, alpha=SMOOTH_ALPHA):
    if key in positions:
        prev_x, prev_y = positions[key]
        x = int(alpha * x + (1 - alpha) * prev_x)
        y = int(alpha * y + (1 - alpha) * prev_y)
    positions[key] = (x, y)
    return x, y

def calculate_palm_center(landmarks):
    indices = [0, 1, 5, 9, 13, 17]
    x = int(np.mean([landmarks[i][0] for i in indices]))
    y = int(np.mean([landmarks[i][1] for i in indices]))
    return x, y

def normalized_dist(a, b, reference):
    d = math.hypot(a[0]-b[0], a[1]-b[1])
    return d / (reference + 1e-6)

def color_by_angle(angle):
    if -45 <= angle <= 45:
        return (0,255,0)
    elif 45 < angle <= 135 or -135 <= angle < -45:
        return (255,0,0)
    else:
        return (0,0,255)


In [6]:
def recognize_by_pattern(finger_states):
    for name, pattern in GESTURE_PATTERNS.items():
        if all(finger_states.get(f, "") == state for f, state in pattern.items()):
            return name
    return ""


In [7]:
def draw_gesture_label(image, palm_x, palm_y, gesture, confidence_count):
    overlay = image.copy()
    x1, y1 = palm_x - 110, palm_y - 80
    x2, y2 = palm_x + 110, palm_y - 20
    cv2.rectangle(overlay, (x1, y1), (x2, y2), VIOLET, -1)
    cv2.addWeighted(overlay, 0.6, image, 0.4, 0, image)
    cv2.putText(image, gesture, (palm_x - 100, palm_y - 35),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,255), 3, cv2.LINE_AA)
    bar_total_w = (x2 - x1) - 20
    bar_h = 10
    bar_x = x1 + 10
    bar_y = y2 - 15
    fill_w = int((confidence_count / GESTURE_HISTORY) * bar_total_w)
    cv2.rectangle(image, (bar_x, bar_y), (bar_x + fill_w, bar_y + bar_h), (0,255,0), -1)
    cv2.rectangle(image, (bar_x, bar_y), (bar_x + bar_total_w, bar_y + bar_h), (255,255,255), 2)

def draw_finger_skeleton(image, landmarks, mcp, pip, tip, state):
    cv2.line(image, tuple(landmarks[mcp]), tuple(landmarks[pip]), JOINT_COLOR[state], 4)
    cv2.line(image, tuple(landmarks[pip]), tuple(landmarks[tip]), JOINT_COLOR[state], 4)
    for p in (mcp, pip, tip):
        cv2.circle(image, tuple(landmarks[p]), 6, JOINT_COLOR[state], -1)

def draw_finger_label(image, tip_x, tip_y, hand_label, finger, state, hand_color):
    line1_pos = (tip_x + 10, tip_y - 25)
    line2_pos = (tip_x + 10, tip_y - 5)
    cv2.line(image, (tip_x, tip_y), (line1_pos[0], line1_pos[1]-5), hand_color, 2)
    cv2.putText(image, f"{hand_label} {finger}", line1_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.6, JOINT_COLOR[state], 2)
    cv2.putText(image, f"{state}", line2_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.6, JOINT_COLOR[state], 2)

def draw_arc(image, center, tip, color):
    dx, dy = tip[0] - center[0], tip[1] - center[1]
    angle = math.degrees(math.atan2(dy, dx))
    radius = max(10, int(math.hypot(dx, dy) * 0.45))
    start_angle = int(angle - 22)
    end_angle = int(angle + 22)
    cv2.ellipse(image, center, (radius, radius), 0, start_angle, end_angle, color, 2)


In [8]:
gesture_histories = {"Left": deque(maxlen=GESTURE_HISTORY), "Right": deque(maxlen=GESTURE_HISTORY)}

def process_hand(image, hand_landmarks, hand_label, smoothed_positions, mode):
    h, w, _ = image.shape
    hand_color = HAND_COLORS.get(hand_label, (200,200,200))
    landmarks = [[int(lm.x * w), int(lm.y * h)] for lm in hand_landmarks.landmark]
    palm_x, palm_y = calculate_palm_center(landmarks)
    palm_x, palm_y = smooth_position(f"{hand_label}_palm", palm_x, palm_y, smoothed_positions)
    ref_spread = math.hypot(landmarks[5][0]-landmarks[17][0], landmarks[5][1]-landmarks[17][1])
    finger_joints = {"Thumb": (2, 3, 4),"Index": (5, 6, 8),"Middle": (9, 10, 12),"Ring": (13, 14, 16),"Pinky": (17, 18, 20)}
    finger_states = {}
    tip_positions = {}
    for finger, (mcp, pip, tip) in finger_joints.items():
        angle = calculate_angle(landmarks[mcp], landmarks[pip], landmarks[tip])
        state = finger_state(finger, angle)
        finger_states[finger] = state
        tip_x, tip_y = smooth_position(f"{hand_label}_{finger}", landmarks[tip][0], landmarks[tip][1], smoothed_positions)
        tip_positions[finger] = (tip_x, tip_y)
    if mode == MODE_NORMAL:
        for finger, (mcp, pip, tip) in finger_joints.items():
            draw_finger_skeleton(image, landmarks, mcp, pip, tip, finger_states[finger])
            tip_x, tip_y = tip_positions[finger]
            draw_finger_label(image, tip_x, tip_y, hand_label, finger, finger_states[finger], hand_color)
    elif mode == MODE_ANGLE:
        for finger in finger_joints:
            tip = tip_positions[finger]
            dx = tip[0] - palm_x; dy = tip[1] - palm_y
            finger_angle = np.degrees(np.arctan2(dy, dx))
            angle_color = color_by_angle(finger_angle)
            cv2.line(image, (palm_x, palm_y), (tip[0], tip[1]), angle_color, 3)
            draw_arc(image, (palm_x, palm_y), (tip[0], tip[1]), angle_color)
            cv2.putText(image, f"{finger_angle:.0f}°", (tip[0] - 15, tip[1] - 25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
    elif mode == MODE_GESTURE:
        candidate = recognize_by_pattern(finger_states)
        ok_candidate = False
        if candidate == "Okay 👌":
            thumb_tip = tip_positions["Thumb"]
            index_tip = tip_positions["Index"]
            if normalized_dist(thumb_tip, index_tip, ref_spread) < DIST_OK_RATIO:
                ok_candidate = True
            else:
                candidate = ""
        if candidate == "Thumbs Up 👍":
            thumb_tip = tip_positions["Thumb"]
            dy = palm_y - thumb_tip[1]
            if dy < ref_spread * 0.15:
                candidate = ""
        hist = gesture_histories[hand_label]
        hist.append(candidate)
        if len(hist) > 0:
            most_common, count = Counter(hist).most_common(1)[0]
        else:
            most_common, count = "", 0
        if most_common != "" and count >= GESTURE_CONFIRM:
            draw_gesture_label(image, palm_x, palm_y, most_common, count)
        else:
            if candidate:
                alpha_overlay = image.copy()
                cv2.putText(alpha_overlay, candidate, (palm_x - 80, palm_y - 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (220,220,220), 2, cv2.LINE_AA)
                cv2.addWeighted(alpha_overlay, 0.25, image, 0.75, 0, image)
    wrist = landmarks[0]
    wrist_x, wrist_y = smooth_position(f"{hand_label}_wrist", wrist[0], wrist[1], smoothed_positions)
    if mode != MODE_ANGLE:
        cv2.putText(image, f"{hand_label} Hand", (wrist_x - 30, wrist_y - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, hand_color, 2, cv2.LINE_AA)


In [9]:
def draw_help_overlay(img):
    hh, ww = img.shape[:2]
    overlay = img.copy()
    box_w, box_h = 420, 140
    x1, y1 = 10, hh - box_h - 10
    cv2.rectangle(overlay, (x1, y1), (x1+box_w, y1+box_h), (20,20,20), -1)
    cv2.addWeighted(overlay, 0.6, img, 0.4, 0, img)
    cv2.putText(img, "Modes: m = switch  |  q = quit", (x1+10, y1+25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1)
    cv2.putText(img, "Normal: skeleton + Straight/Bent", (x1+10, y1+50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), 1)
    cv2.putText(img, "Gesture: stabilized (shows when stable)", (x1+10, y1+75), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), 1)
    cv2.putText(img, "Angle: fingertip angle from palm (°)", (x1+10, y1+100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), 1)
    cv2.putText(img, "Recognized gestures: Fist, Open Hand, Peace, Thumbs Up, Pointing, Okay, Rock, Call Me", (x1+10, y1+125),
                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (180,180,180), 1)


In [10]:
def main():
    global mode
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    smoothed_positions = {}
    prev_time = 0
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose, \
         mp_hands.Hands(min_detection_confidence=0.6, min_tracking_confidence=0.6, max_num_hands=2) as hands:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.flip(frame, 1)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            pose_results = pose.process(image)
            hand_results = hands.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if pose_results.pose_landmarks:
                mp_drawing.draw_landmarks(image, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            if hand_results.multi_hand_landmarks:
                for idx, hand_landmarks in enumerate(hand_results.multi_hand_landmarks):
                    hand_label = hand_results.multi_handedness[idx].classification[0].label
                    process_hand(image, hand_landmarks, hand_label, smoothed_positions, mode)
            curr_time = time.time()
            fps = 1 / (curr_time - prev_time) if prev_time else 0
            prev_time = curr_time
            cv2.putText(image, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2)
            mode_text = "Normal Mode" if mode == MODE_NORMAL else "Gesture Mode" if mode == MODE_GESTURE else "Angle Mode"
            cv2.putText(image, f"Mode: {mode_text} (Press 'm' to switch)", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)
            draw_help_overlay(image)
            cv2.imshow('Hand + Finger Detection (Quantum-Gear Styled)', image)
            key = cv2.waitKey(10) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('m'):
                mode = (mode + 1) % 3
                for k in gesture_histories:
                    gesture_histories[k].clear()
    cap.release()
    cv2.destroyAllWindows()


In [11]:
if __name__ == "__main__":
    main()

c:\Users\dibye\OneDrive\Desktop\Gesture Detection\myenv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
